## Created by Diana Janik and Jan Markiewicz

In [167]:
import numpy as np


import matplotlib.pyplot as plt
import matplotlib.text as text

## Linear regression

$$ y_i = x_{ij} w_j + b$$

$$ y_i = x_{ij} w_j, \quad x_{i,-1}=1,\quad b=w_{-1} $$

In [168]:
def linear(x,w):
    return x @ w

Generate a random feature vector $\mathbf{x}$ witch 10000 samples and three feature 
such that first feature is drawn from N(0,1), second feature from  U(,1) and third from N(1,2).

In [169]:
x = np.stack((np.random.normal(0, 1, (1000)), 
              np.random.uniform(0, 1, (1000)), 
              np.random.normal(1,2, (1000))), axis=1)
x.shape

(1000, 3)

N(mu,sigma) denotes normal distribution with mean mu and standard deviation sigma. You can use ``numpy.random.normal`` and ``numpy.random.uniform`` functions.

Using $\mathbf{x}$ and weights w = [0.2, 0.5,-0.25,1.0] generate output $\mathbf{y}$ assuming a $N(0,0.1)$ noise $\mathbf{\epsilon}$. 

In [170]:
w = np.array((0.2, 0.5, -0.25, 1.))
ones = np.ones((x.shape[0], 1))
x = np.concatenate((x,ones), axis = 1)
noise = np.random.normal(0, 0.1)
y = linear(x,w)
y = y + noise

$$ y_i = x_{ij} w_j+\epsilon_i, \quad x_{i,-1}=1,\quad b=w_{-1} $$

#### Loss

$$ \frac{1}{2}\frac{1}{N}\sum_{i=0}^{N-1} (y_i -  x_{ij} w_j  )^2$$

In [171]:
def getLoss(y, x, w):
    loss = np.square(y - linear(x,w))
    loss = np.sum(loss) / (2*y.shape[0])
    return loss


## Gradient descent 

### Problem 1 

Find the gradient of the loss function with respect to weights.

Write gradient function ``grad(y,x,w)``.

In [172]:
def grad(y, x, w):
    diff = (x @ w - y)
    return np.dot(x.T, diff) / x.shape[0]
#     return (w - (alpha/x.shape[0]) * tmp)
gradient = grad(y, x, w)
print(gradient)

[ 0.00254884 -0.07115897 -0.13088844 -0.13759184]


### Problem 2

Implement gradient descent for linear regression.

In [173]:
alpha = 0.1

def gradientDescent(y, x, w, maxIterations = 500, tolerance=0.0000001):
    for i in range(maxIterations):
        loss = getLoss(y, x, w)
        if loss < tolerance:
            maxIterations = i
            break
        gradient = grad(y, x, w)
        w = w - alpha*gradient

    print("finished gradient descent on iteration " + str(maxIterations))
    print("with loss equal " + str(loss))
    return w
        
gradientDescent(y, x, w)

finished gradient descent on iteration 500
with loss equal 1.959934560652158e-07


array([ 0.19997747,  0.50210241, -0.24998088,  1.13640725])

### Problem 3

Implement stochastic gradient descent (SGD).

In [174]:
def getBatches(x, y, batchSize):
    randomIndices = np.random.randint(1000, size=(batchSize))
    xResult = []
    yResult = []
    for i in randomIndices:
        xResult.append(x[i])
        yResult.append(y[i])
    return (np.asanyarray(xResult), np.asanyarray(yResult))

def sgd(y, x, w, maxIterations = 500, tolerance=0.0000001, batchSize = 10):
    for i in range(maxIterations):
        loss = getLoss(y, x, w)
        if loss < tolerance:
            maxIterations = i
            break
        randomIndices = np.random.randint(1000, size=(batchSize))
        selectedX = x[randomIndices]
        selectedY = y[randomIndices]
#         (selectedX, selectedY) = getBatches(x, y, batchSize)
        gradient = grad(selectedY, selectedX, w)
        w = w - alpha*gradient

    print("finished gradient descent on iteration " + str(maxIterations))
    print("with loss equal " + str(loss))
    return w
sgd(y, x, w)

finished gradient descent on iteration 500
with loss equal 1.929119346193321e-07


array([ 0.20002551,  0.50199272, -0.2498733 ,  1.13648186])

In [175]:
print("SGD takes: ")
%time tSGD = sgd(y, x, w)
print("gradient descent takes: ")
%time tGD = gradientDescent(y, x, w)

SGD takes: 
finished gradient descent on iteration 500
with loss equal 2.0815189685419418e-07
CPU times: user 32.1 ms, sys: 160 µs, total: 32.3 ms
Wall time: 31.5 ms
gradient descent takes: 
finished gradient descent on iteration 500
with loss equal 1.959934560652158e-07
CPU times: user 15.4 ms, sys: 0 ns, total: 15.4 ms
Wall time: 15.3 ms


### Problem 4

Implement SGD using pytorch. Start by just rewritting Problem 3 to use torch Tensors instead of numpy arrays. 

To convert frrom numpy arrays to torch tensors you can use ``torch.from_numpy()`` function. 

In [176]:
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

def modelTorch(x):
    return x @ w.t() + b

In [177]:
inputs = np.stack((np.random.normal(0, 1, (1000)), 
              np.random.uniform(0, 1, (1000)), 
              np.random.normal(1,2, (1000))), axis=1)
inputs = torch.from_numpy(inputs).float().to(device)

targets = model(inputs)
targets = np.asanyarray([[float(x + noise)] for x in targets])
targets = torch.from_numpy(targets).float()

w = torch.randn(1, 3, requires_grad=True)
b = torch.randn(1, requires_grad=True)

train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)
next(iter(train_dl))

[tensor([[-0.3724,  0.6897,  1.8082],
         [-0.9042,  0.5787, -1.0603],
         [ 0.8851,  0.4437,  1.1528],
         [ 0.0732,  0.6072,  1.4597],
         [-0.5207,  0.4177,  4.3378]]), tensor([[-0.4173],
         [ 1.1055],
         [-0.7998],
         [-0.4994],
         [-1.5507]])]

In [178]:
model = nn.Linear(3, 1)
opt = torch.optim.SGD(model.parameters(), lr=1e-5)
loss_fn = F.mse_loss

In [179]:
num_epochs = 100
tolerance = 0.01
for epoch in range(num_epochs):
        loss_total = 0
        for xb,yb in train_dl:
            pred = model(xb)
            loss = loss_fn(pred, yb)
            loss_total += loss
            loss.backward()
            opt.step()
            opt.zero_grad()
        loss_avg = loss_total/(len(train_dl))
        print(loss_avg)
        if loss_avg < tolerance:
            break
print('Training loss: ', loss_fn(model(inputs), targets))

tensor(4.6237, grad_fn=<DivBackward0>)
tensor(4.4343, grad_fn=<DivBackward0>)
tensor(4.2526, grad_fn=<DivBackward0>)
tensor(4.0785, grad_fn=<DivBackward0>)
tensor(3.9115, grad_fn=<DivBackward0>)
tensor(3.7515, grad_fn=<DivBackward0>)
tensor(3.5980, grad_fn=<DivBackward0>)
tensor(3.4508, grad_fn=<DivBackward0>)
tensor(3.3097, grad_fn=<DivBackward0>)
tensor(3.1745, grad_fn=<DivBackward0>)
tensor(3.0448, grad_fn=<DivBackward0>)
tensor(2.9204, grad_fn=<DivBackward0>)
tensor(2.8012, grad_fn=<DivBackward0>)
tensor(2.6869, grad_fn=<DivBackward0>)
tensor(2.5773, grad_fn=<DivBackward0>)
tensor(2.4722, grad_fn=<DivBackward0>)
tensor(2.3714, grad_fn=<DivBackward0>)
tensor(2.2748, grad_fn=<DivBackward0>)
tensor(2.1822, grad_fn=<DivBackward0>)
tensor(2.0934, grad_fn=<DivBackward0>)
tensor(2.0082, grad_fn=<DivBackward0>)
tensor(1.9266, grad_fn=<DivBackward0>)
tensor(1.8483, grad_fn=<DivBackward0>)
tensor(1.7733, grad_fn=<DivBackward0>)
tensor(1.7013, grad_fn=<DivBackward0>)
tensor(1.6323, grad_fn=<D

### Problem 5 

Implement GD using pytorch automatic differentiation.

To this end the variable with respect to which the gradient will be calculated, ``t_w`` in this case, must have attribute
``requires_grad`` set to ``True`` (``t_w.require_grad=True``).

The torch will automatically track any expression containing ``t_w`` and store its computational graph. The method ``backward()`` can be run on the final expression to back propagate the gradient e.g. ``loss.backward()``. Then the gradient is accesible as ``t_w.grad``.